# Dataanalyse for Ingerslev Maskinfabrik
Indsigt i hvad og hvor meget vi får lavet hos Ingerslev, samt hvorvidt produktionen kan lægges ud til Indien

Loading in the data

In [19]:
import pandas as pd
import openpyxl
import numpy as np

# Load the data
file_path = "C:/Users/sgj/OneDrive - Alfotech/Skrivebord/Alfotech/BI project/Vareposter Alfotech.xlsx"
vareposter = pd.read_excel(file_path, header=0)

In [20]:
# Remap Montageforbrug → Salg in-place
vareposter['Posttype'] = vareposter['Posttype'].replace({'Montageforbrug': 'Salg'})

In [21]:
vareposter.head()  # Display the first few rows of the dataframe to verify the changes

,Bogføringsdato,Posttype,Bilagstype,Bilagsnr.,Varenr.,Beskrivelse,Lokationskode,Antal,Faktureret antal,Restantal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Kostbeløb (ikke-lager),Åben,Ordretype,Løbenr.,Kildetype,Kildenr.,Leverandørnr.
0,2025-04-30,Køb,Købsleverance,20924732,010153492910,NaN,LAGER,4.0,4.0,0.0,0.00,434.16,0.0,False,NaN,228867,Kreditor,86243311,86243311
1,2025-04-30,Salg,Salgsleverance,112152,0278021003,"Push-in, male connector, AISI 316,",LAGER,-4.0,-4.0,0.0,595.75,-204.44,0.0,False,NaN,228646,Debitor,GB35316289530,IT2661006
2,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.83,-99.92,0.0,False,NaN,228645,Debitor,GB35316289530,IT307401249
3,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228644,Debitor,GB35316289530,IT307401249
4,2025-04-30,Salg,Salgsleverance,112152,04211S1008,"Teflon hose, smooth, FDA & DVGW,",LAGER,-5.0,-5.0,0.0,359.84,-99.92,0.0,False,NaN,228643,Debitor,GB35316289530,IT307401249


Isolating Ingerslev as supplier for this analysis

In [91]:
ingerslev_df = vareposter[vareposter['Leverandørnr.'] == '87371811']

In [25]:
ingerslev_df.head()  # Display the first few rows of the Ingerslev dataframe to verify the changes

,Bogføringsdato,Posttype,Varenr.,Beskrivelse,Antal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Leverandørnr.
26,2025-04-30,Salg,2080009102,NaN,-1.0,0.0,-107.00,87371811
27,2025-04-30,Salg,2080009101,NaN,-1.0,0.0,-35.00,87371811
66,2025-04-30,Salg,0463501204,NaN,-4.0,1800.0,-974.29,87371811
68,2025-04-30,Salg,2020091002,NaN,-10.0,0.0,-780.00,87371811
69,2025-04-30,Salg,2020091001,NaN,-10.0,0.0,-730.00,87371811


Dividing into Køb and Salg dataframes

In [26]:
ingerslev_køb = ingerslev_df[ingerslev_df['Posttype'] == 'Køb']
ingerslev_salg = ingerslev_df[ingerslev_df['Posttype'] == 'Salg']

Understanding the relationship between how much we sell and how often we buy.

In [55]:
from IPython.display import Markdown, display

display(Markdown("### Frequencies of Køb and Salg:"))

display(
    ingerslev_df['Posttype']
    .value_counts()
    .rename_axis(None)  # Fjerner index-navn
    .rename(None)       # Fjerner kolonnenavn
)


### Frequencies of Køb and Salg:

Salg    1004
Køb      126
dtype: int64

In [57]:
# Tæl rækker i hver DataFrame
num_køb = len(ingerslev_køb)
num_salg = len(ingerslev_salg)

# Beregn forholdet
forhold = num_salg / num_køb
print(f"For hver {forhold:.2f} salg sker der ét køb.")

For hver 7.97 salg sker der ét køb.


## Produktniveau analyse
Hvor høj en frekvens af salgs- og købsordrer vi har. Antal solgte og købte produkter vil være pr. ordre. Dette håndteres senere i analysen. 

In [95]:
# Højeste frekvens af salgsordrer
frequent_orders_salg = ingerslev_salg['Varenr.'].sort_values(ascending=False)
print(f"Varenumre med flest salgsordrer: \n{frequent_orders_salg.
                                value_counts().
                                head(10).
                                rename_axis(None).
                                rename(None)}")

Varenumre med flest salgsordrer: 
2020091001    123
2020091002    120
0101651510     66
2020091303     33
2020091302     29
0463506417     26
0463502505     23
0463501217     17
0494025036     16
2020091305     16
dtype: int64


In [96]:
# Højeste frekvens af købsordrer
frequent_orders_køb = ingerslev_køb['Varenr.'].sort_values(ascending=False)
print(f"Varenumre med flest købsordrer: \n{frequent_orders_køb.
                                value_counts().
                                head(10).
                                rename_axis(None).
                                rename(None)}")

Varenumre med flest købsordrer: 
2069309025      15
0463506417       5
0101651510       4
1101105011R      4
2020091001       3
1190148150       3
2020091002       3
0499064094       3
2080009101       3
04635032032S     3
dtype: int64


In [ ]:
ingerslev_df.head()

,Bogføringsdato,Posttype,Bilagstype,Bilagsnr.,Varenr.,Beskrivelse,Lokationskode,Antal,Faktureret antal,Restantal,Salgsbeløb (faktisk),Kostbeløb (faktisk),Kostbeløb (ikke-lager),Åben,Ordretype,Løbenr.,Kildetype,Kildenr.,Leverandørnr.
26,2025-04-30,Salg,Bogført montage,A20866,2080009102,NaN,LAGER,-1.0,-1.0,0.0,0.0,-107.00,0.0,False,Montage,228619,Vare,0280009102,87371811
27,2025-04-30,Salg,Bogført montage,A20866,2080009101,NaN,LAGER,-1.0,-1.0,0.0,0.0,-35.00,0.0,False,Montage,228618,Vare,0280009102,87371811
66,2025-04-30,Salg,Salgsleverance,112129,0463501204,NaN,LAGER,-4.0,-4.0,0.0,1800.0,-974.29,0.0,False,NaN,228573,Debitor,28947867,87371811
68,2025-04-30,Salg,Bogført montage,A20861,2020091002,NaN,LAGER,-10.0,-10.0,0.0,0.0,-780.00,0.0,False,Montage,228570,Vare,0469009100,87371811
69,2025-04-30,Salg,Bogført montage,A20861,2020091001,NaN,LAGER,-10.0,-10.0,0.0,0.0,-730.00,0.0,False,Montage,228569,Vare,0469009100,87371811


### Følgende omhandler antal varer vi sælger og køber i en given periode. 

In [ ]:
import pandas as pd

# Sørg for, at 'Bogføringsdato' er datetime
df = ingerslev_df.copy()
df['Bogføringsdato'] = pd.to_datetime(df['Bogføringsdato'], dayfirst=True)
df = df.sort_values(['Varenr.', 'Bogføringsdato'])

resultater = []

for varenr, gruppe in df.groupby('Varenr.'):
    køb = gruppe[gruppe['Faktureret antal'] > 0].reset_index(drop=True)
    
    for i in range(len(køb) - 1):
        start = køb.loc[i, 'Bogføringsdato']
        slut = køb.loc[i + 1, 'Bogføringsdato']
        indkøbt_antal = køb.loc[i, 'Faktureret antal']
        
        # Udtræk salg i perioden mellem disse to køb
        salg_i_interval = gruppe[
            (gruppe['Bogføringsdato'] > start) &
            (gruppe['Bogføringsdato'] <= slut) &
            (gruppe['Faktureret antal'] < 0)
        ]
        
        antal_salg = len(salg_i_interval)
        solgt_antal = -salg_i_interval['Faktureret antal'].sum()  # vend minus til plus
        dage_mellem_køb = (slut - start).days
        
        omsætningshastighed = (
            solgt_antal / indkøbt_antal
            if indkøbt_antal > 0 else None
        )
        
        resultater.append({
            'Varenr.': varenr,
            'Startdato': start,
            'Slutdato': slut,
            'Indkøbt antal': indkøbt_antal,
            'Antal salg': antal_salg,
            'Solgt antal': solgt_antal,
            'Dage mellem køb': dage_mellem_køb,
            'Omsætningshastighed': omsætningshastighed
        })

# Konverter til DataFrame
resultat_df = pd.DataFrame(resultater)


In [107]:
resultat_df.head(10)

,Varenr.,Startdato,Slutdato,Indkøbt antal,Antal salg,Solgt antal,Dage mellem køb,Omsætningshastighed
0,0101600380,2023-05-08,2024-12-19,13.0,3,7.0,591,0.538462
1,0101651510,2023-05-01,2023-09-25,22.0,14,22.0,147,1.000000
2,0101651510,2023-09-25,2024-03-15,34.0,15,32.0,172,0.941176
3,0101651510,2024-03-15,2024-04-10,2.0,4,5.0,26,2.500000
4,0101651510,2024-04-10,2024-06-21,17.0,11,15.0,72,0.882353
5,0101651510,2024-06-21,2024-12-19,30.0,16,20.0,181,0.666667
6,0102271100A,2023-10-05,2024-02-06,2.0,1,2.0,124,1.000000
7,0102273100A,2023-10-05,2024-02-06,2.0,1,2.0,124,1.000000
8,0102273150A,2024-02-06,2024-02-06,24.0,0,-0.0,0,-0.000000
9,0280019101,2025-01-21,2025-02-06,2.0,0,-0.0,16,-0.000000


In [124]:
# Unikke varenumre
unikke_varer_df = pd.DataFrame(ingerslev_df["Varenr."].drop_duplicates().reset_index(drop=True))
unikke_varer_df = unikke_varer_df[["Varenr."]]  # Omarranger kolonnerne
display(unikke_varer_df)

,Varenr.
0,2080009102
1,2080009101
2,0463501204
3,2020091002
4,2020091001
...,...
148,0494076090
149,0493060761
150,2020091005
151,0101630760
